In [21]:
import pickle
import pandas as pd
import os
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

In [2]:
print(os.listdir("../wine-quality"))

['.git', '.ipynb_checkpoints', 'app.py', 'classification_data.pkl', 'classification_label.pkl', 'classification_model.pkl', 'classification_test_data.pkl', 'classification_test_label.pkl', 'Combine_Of_XGBOOST_NN.ipynb', 'regression_data.pkl', 'regression_label.pkl', 'regression_model.pkl', 'regression_test_data.pkl', 'regression_test_label.pkl', 'train.ipynb', 'winequality-red.csv']


In [3]:
with open('classification_data.pkl', 'rb') as file:
    data = pickle.load(file)
with open('classification_label.pkl', 'rb') as file:
    label = pickle.load(file)
with open('classification_test_data.pkl', 'rb') as file:
    data_test = pickle.load(file)
with open('classification_test_label.pkl', 'rb') as file:
    label_test = pickle.load(file)

In [4]:
with open('classification_model.pkl', 'rb') as file:
    # Load the data from the file
    xgb = pickle.load(file)

In [5]:
xgb.fit(data, label)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.8, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

Extract the feature weights learned by the XGBoost model

In [6]:
xgb_features = xgb.feature_importances_

Use the feature weights as inputs to a neural network

In [7]:
nn_model = MLPClassifier()
nn_model.fit(data, label)

C:\Users\Gökçe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

StackingClassifier that uses the predictions from the XGBoost and neural network models as features for a final model

In [8]:
stacking_model = StackingClassifier(estimators=[('xgb', xgb), ('nn', nn_model)], final_estimator=XGBClassifier())

In [9]:
stacking_model.fit(data, label)

C:\Users\Gökçe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Gökçe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Gökçe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Gökçe\AppData

StackingClassifier(estimators=[('xgb',
                                XGBClassifier(base_score=0.5, booster='gbtree',
                                              callbacks=None,
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None, gamma=0,
                                              gpu_id=-1,
                                              grow_policy='depthwise',
                                              importance_type=None,
                                              interaction_constraints='',
                                              learning_rate=0.8, max_bin=256,
                                              max_...
                                                 gpu_id=None, grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=100, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 predictor=None,
                                                 random_state=None,
                                                 reg_alpha=None,
                                                 reg_lambda=None, ...))

In [10]:
predictions = stacking_model.predict(data_test)

In [11]:
xgb_predictions = xgb.predict(data_test)

In [12]:
mlp_model = MLPClassifier(hidden_layer_sizes=(128,), max_iter=10, activation='relu')

In [13]:
mlp_model.fit(data, label)

C:\Users\Gökçe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(128,), max_iter=10)

In [14]:
mlp_predictions = mlp_model.predict(data_test)

In [15]:
accuracy = accuracy_score(label_test, predictions)
print('Accuracy score of '+ stacking_model.__class__.__name__ + ":" +str(accuracy))

Accuracy score of StackingClassifier:0.984


In [16]:
accuracy_mlp = accuracy_score(label_test, mlp_predictions)
print('Accuracy score of '+ mlp_model.__class__.__name__ + ":" +str(accuracy_mlp))

Accuracy score of MLPClassifier:0.802


In [17]:
accuracy_xgb = accuracy_score(label_test, xgb_predictions)
print('Accuracy score of '+ xgb.__class__.__name__ + ":" +str(accuracy_xgb))

Accuracy score of XGBClassifier:0.972


In [18]:
loss = mean_absolute_error(label_test, predictions)
print('Loss of '+ stacking_model.__class__.__name__ + ":" +str(loss))

Loss of StackingClassifier:0.016


In [19]:
loss_mlp = mean_absolute_error(label_test, mlp_predictions)
print('Loss of '+ mlp_model.__class__.__name__ + ":" +str(loss_mlp))

Loss of MLPClassifier:0.198


In [20]:
loss_xgb = mean_absolute_error(label_test, xgb_predictions)
print('Loss of '+ xgb.__class__.__name__ + ":" +str(loss_xgb))

Loss of XGBClassifier:0.028


In [22]:
Model = ['Neural Network with XGBOOST','Neural Network without XGBOOST','XGBClassifier']
value = []
for i in [accuracy, accuracy_mlp, accuracy_xgb]:
    value.append(i)
    
data = {'Models': Model , 'Accuracy':  value }
temp = pd.DataFrame(data)

In [23]:
temp

,Models,Accuracy
0,Neural Network with XGBOOST,0.984
1,Neural Network without XGBOOST,0.802
2,XGBClassifier,0.972


In [24]:
value = []
for i in [loss, loss_mlp, loss_xgb]:
    value.append(i)
    
data = {'Models': Model , 'Loss':  value }
temp = pd.DataFrame(data)

In [25]:
temp

,Models,Loss
0,Neural Network with XGBOOST,0.016
1,Neural Network without XGBOOST,0.198
2,XGBClassifier,0.028
